In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/customer/customer.csv")

In [ ]:
df.head()

<p>Catergorical Data is - object type</p>
<p>we convert it into number before training the model</p>

<h3>Approach according to need</h3>
<p>Two types of data - Nominal (no order), Ordinal (with order)</p>
<p>Example of Nominal - Weather (Rainy, Cloudy, Sunny)</p>
<p>Example of Ordinal - Experience (Good > Okish > Bad)</p>

<p>For input ordinal data - OrdinalEncoder(), for output ordinal data - LabelEncoder()</p>
<p>For Nominal input and output data - OneHotEncoding()</p>
<p><b>First lets learn to handle Ordinal Data, and ignore the Nominal Data<b></p>

In [ ]:
ordinal_output_col = ['purchased']
ordinal_input_cols = ['review','education'];
ordinal_input_cols

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df.iloc[:,2:4],df.iloc[:,-1],test_size=0.2)

In [ ]:
X_train.head()

<p>Ordinal Encoding on 'review' ,'education'</p>
<p>Label Encoding on 'purchased'</p>

<h3>Ordinal Encoding</h3>
<p>Notice how we only feed the object ordinal columns</p>

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
oe = OrdinalEncoder(categories=[['Poor','Average','Good'],['School','UG','PG']])
label_X_train[ordinal_input_cols] = oe.fit_transform(X_train[ordinal_input_cols])
label_X_valid[ordinal_input_cols] = oe.transform(X_valid[ordinal_input_cols])

In [ ]:
label_X_train.sample(5)

<h3>Label Encoding</h3>
<p>this converts y_train and y_valid to numpy array</p>

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder() #cannot decide order here, it will be randomly assigned

# Make copy to avoid changing original data 
label_y_train = y_train.copy()
label_y_valid = y_valid.copy()

# Apply label encoder to output column
label_y_train = le.fit_transform(y_train)
label_y_valid = le.transform(y_valid)

In [ ]:
label_y_train

In [ ]:
print("Score after doing Ordinal and Nominal Encoding") 
print(f'{score_dataset(label_X_train, label_X_valid, label_y_train, label_y_valid):.2f}')

<h1>One Hot Encoding</h1>
<p>Now that we are done with Ordinal and Label encoders, we will see OHE with a <b>fresh new dataset</b></p>

In [ ]:
data = pd.read_csv("/kaggle/input/cars-dataset/cars.csv");

In [ ]:
data.head()

In [ ]:
data.nunique()

<h2>OneHotEncoding using Pandas</h2>
<p>OneHotEncoding can be performed using pandas and sklearn. However, pandas does not retain the order of columns, so it is not commonly used for this purpose</p>

<p>In Machine Learning, it is important to ensure that your columns are not dependent on or have mathematical relationships with each other, which is known as <em>multicollinearity</em>. Hence, your columns should not exhibit multicollinearity.

To address multicollinearity when using OneHotEncoding, we remove one column. If you have 'n' categories in a column, after OneHotEncoding, you will have 'n-1' columns remaining. These resulting columns from OneHotEncoding are referred to as <em>dummy variables</em>. The creation of these dummy variables introduces multicollinearity, which is known as the <em>"dummy variable trap."</em> To avoid falling into this trap, we remove one column.</p>

In [ ]:
#we will not use this for brand as of now, we will deal with large categories colums later ahead
pd.get_dummies(data, columns = ['fuel','owner'], drop_first=True)

<h2>OneHotEncoding using Sklearn</h2>
<p>OneHotEncoding has several parameters. The "sparse" parameter determines whether the output will be a dataframe (if True) or a numpy array (if False). The "drop" parameter, when set to "first," drops the first column. Additionally, the "dtype" parameter, when set to "numpy integer," performs OneHotEncoding with 0 and 1 integers instead of floats.</p>

In [ ]:
my_cols = ['fuel','owner']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.iloc[:,0:4],data.iloc[:,-1],test_size=0.2,random_state=2)
X_train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)
X_train_new = ohe.fit_transform(X_train[my_cols]) #use .values if sparse=True
X_test_new = ohe.transform(X_test[my_cols])

#add all featues together
final_X_train = np.hstack((X_train[['brand','km_driven']].values,X_train_new))

In [ ]:
X_train_new

In [ ]:
final_X_train

<h3>now handle columns of high categories</h3>

In [ ]:
counts = data['brand'].value_counts()
counts

In [ ]:
threshold = 100
data['brand'].nunique()

In [ ]:
replace_cols = counts[counts <= threshold].index
replace_cols

In [ ]:
OHE_high_cat_col = pd.get_dummies(data['brand'].replace(replace_cols, 'uncommon'))
OHE_high_cat_col.head()